In [8]:
pip install bitarray


In [9]:
import numpy as np
from bitarray import bitarray
from google.colab import files
uploaded = files.upload()

Saving security_file.txt to security_file (1).txt


**Read Text File**

**Split into Blocks of 64-bits**

In [14]:
test_str = "security_file.txt"
# read file as string:
f = open(test_str, 'r')
text = f.read()
#Return the binary value as a string of the given size 
def char_size(val, bitsize): 
    char_byte = bin(val)[2:] if isinstance(val, int) else bin(ord(val))[2:]
    if len(char_byte) > bitsize:
        raise "binary value larger than the expected size"
    while len(char_byte) < bitsize:
        char_byte = "0"+char_byte #Add as many 0 as needed to get the wanted size
    return char_byte
#Convert a string into a list of bits
def string_to_bit_array(text):
    array = list()
    for char in text:
        char_byte = char_size(char, 8)#Get the char value on one byte
        array.extend([int(x) for x in list(char_byte)]) #Add the bits to the final list
    return array
array=string_to_bit_array(text)
array=np.array(array)
#Recreate the string from the bit array
def bit_array_to_string(array): 
    res = ''.join([chr(int(y,2)) for y in [''.join([str(x) for x in _bytes]) for _bytes in  nsplit(array,8)]])   
    return res


def addPadding(X):
    if len(X) % 64 == 0:
       return X
    else:
        len_Bin_file=len(array)
        print("length of bin file"+ " "+str(len_Bin_file))
        num=len_Bin_file//64
        num_2=len_Bin_file/64 #output the remainder in decimal 
        rest_of_bits=len_Bin_file-(num*64) #The size of remainder in bits
        print(str(num)+"  " +str(rest_of_bits))
        rest=array[(num*64)+1:len_Bin_file-1]  #the remainder bits
        extra=np.zeros(64-len(rest),dtype=int)  #padding of zeros on the left
        reem=np.append(rest,extra)
        print(rest)
        print(len(reem))
        array_2=array[0:(num*64)]
        new_array=np.append(array_2,reem)
        s=len(new_array)
    return new_array
#Split a list into sublists of size "n"
def nsplit(s, n):
    return [s[k:k+n] for k in range(0, len(s), n)]

    
# #Add padding to the datas using PKCS5 spec.
# def addPadding(self):
#         pad_len = 8 - (len(self.text) % 8)
#         self.text += pad_len * chr(pad_len)
        
#         return self.text

new_array=addPadding(array)
IP_input=nsplit(new_array,64)
print(IP_input[-1])

length of bin file 28328
442  40
[1 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0 1 1 1 0 1 0 0 0 1 1 1 0 1 0 1 0 0 1 1 1 1
 1]
64
[1 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0 1 1 1 0 1 0 0 0 1 1 1 0 1 0 1 0 0 1 1 1 1
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]



**Key Transformation**

In [15]:
#the initial generated key
K =[0,0,0,1,0,0,1,1,0,0,1,1,0,1,0,0,0,1,0,1,0,1,1,1,0,1,1,1,1,0,0,1,1,0,0,1,1,0,1,1,1,0,1,1,1,1,0,0,1,1,0,1,1,1,1,1,
1,1,1,1,0,0,0,1]  

keys=bitarray() 


#Initial permut made on the key

PC_1 = [57, 49, 41, 33, 25, 17, 9,
        1, 58, 50, 42, 34, 26, 18,
        10, 2, 59, 51, 43, 35, 27,
        19, 11, 3, 60, 52, 44, 36,
        63, 55, 47, 39, 31, 23, 15,
        7, 62, 54, 46, 38, 30, 22,
        14, 6, 61, 53, 45, 37, 29,
        21, 13, 5, 28, 20, 12, 4]

PC_2=[14,17,11,24 ,1 ,5,3 ,28, 15, 6, 21, 10,23, 19, 12, 4, 26, 8,16, 7, 27, 20 ,13, 2,41 ,52, 31, 37, 47, 55,30 ,40 ,51, 45, 33, 48,
44 ,49 ,39, 56, 34, 53,
46 ,42 ,50, 36, 29, 32]

SHIFT = [1,1,2,2,2,2,2,2,1,2,2,2,2,2,2,1]

def keySplit(key):
    c=key[:28]
    d=key[28:]
    return c , d


def permute(k, table):#Permut the given block using the given table (so generic method)
    New_M = ""  # To add in it the rearranged bits of Input_M
  # add in New_M the value in Input_M[i] (ex: Input_M[58])
  # where the i comes from the P_Box_Table/IP_Table/FP_Table and the - 1 due to starting from 0 index
    for i in table:
		    New_M += str(k[int(i)-1])
  
    return New_M
    
#Matrix that determine the shift for each round of keys

def shift(C0,D0,n):
  temp=C0[:n]
  C0=C0[n:] + temp
  temp2=D0[:n]
  D0=D0[n:]+temp2
  return C0,D0 

New_key = np.empty(0,dtype=np.int8)

  
def KeyTransformation(key):
  keys=[]
  k=permute(key,PC_1)
  c,d=keySplit(k)
  for i in range(16):
      c , d =shift(c,d,SHIFT[i])
      cidi=c + d
      keys.append(permute(cidi,PC_2))
  return keys

def FinalKeyTransformation(key):
 keys = KeyTransformation(key)
 fNew_key = np.empty(0,dtype=np.int8)
 for char in keys:
    fNew_key=np.append(fNew_key,(char))

#  print(fNew_key)
#  print(type(fNew_key))
 return fNew_key


In [12]:
Final_Key_Trans = FinalKeyTransformation(K)
#print(Final_Key_Trans)

**Initial Permuation "IP"**

In [17]:

#converting from Hex to Binary

def converthextobinary(hex):
	Binary = ""
	for i in hex:
		Binary += bin(int(i,16))[2:].zfill(4)
	return Binary
  
  #Permutation Function
def PermutationFunction(TABLE, Input_M):
	New_M = ""  # To add in it the rearranged bits of Input_M
  # add in New_M the value in Input_M[i] (ex: Input_M[58])
  # where the i comes from the P_Box_Table/IP_Table/FP_Table and the - 1 due to starting from 0 index
	for i in TABLE:
		New_M += Input_M[int(i)-1]
  
	return New_M

  

 # Intial Permutation Implementation

# This is the given IP Table which is used to rearrange the M bits 

IP_Table = ['58 ', '50 ', '42 ', '34 ', '26 ', '18 ', '10 ', '2',
			 '60 ', '52 ', '44 ', '36 ', '28 ', '20 ', '12 ', '4',
			 '62 ', '54 ', '46 ', '38 ', '30 ', '22 ', '14 ', '6', 
			'64 ', '56 ', '48 ', '40 ', '32 ', '24 ', '16 ', '8', 
			'57 ', '49 ', '41 ', '33 ', '25 ', '17 ', '9 ', '1',
			 '59 ', '51 ', '43 ', '35 ', '27 ', '19 ', '11 ', '3',
			 '61 ', '53 ', '45 ', '37 ', '29 ', '21 ', '13 ', '5',
			 '63 ', '55 ', '47 ', '39 ', '31 ', '23 ', '15 ', '7']

# This is the method where we rearrange bits of M (IP)

def InitialPermute(IP_TABLE, BinaryM):
  P_M = PermutationFunction(IP_Table,BinaryM)
  #print("P_M", P_M)
  return P_M

# Testing the converthextobinary and the InitialPermute Methods

#M =  # 16 digits so 64 bits
#BinaryM = converthextobinary(M)
#print("This is the length of our Input Block:",len(converthextobinary(M)))
#print ("This is the converted to binary message:", BinaryM)
#print ("This is our Initially Permuted Message:", InitialPermute(IP_Table, BinaryM))

# This is the method where we split our permuted msg into left and right 32 bits
def Permute_Split(P_M) :
  LPT=""
  RPT=""
  LPT= P_M[: 32] # Take the left 32 bits
  RPT= P_M[32 :] # Take the right 32 bits
  
  New_RPT = np.empty(0,dtype=np.int8)
  New_LPT = np.empty(0,dtype=np.int8)



  for char in RPT:
    New_RPT = np.append(New_RPT,int(char))

  for char in LPT:
    New_LPT = np.append(New_LPT,int(char))  


  #print(type(New_RPT))
  print("New RPT :", New_RPT)

  #print(type(New_LPT))
  print("New LPT :", New_LPT)

  return New_LPT, New_RPT

# Testing the Permute_Split method
# PermutedMsg=InitialPermute(IP_Table, )
# print("These are the LPT and RPT: ", Permute_Split(PermutedMsg))
 


**Expansion Permutation**

In [20]:
#Split the RPT into Sub-Arrays
# PermutedMsg=InitialPermute(IP_Table, BinaryM)
# LPT , RPT =  Permute_Split(PermutedMsg)

def split_array(New_RPT, a):
    sublists = list()
    for i in range(8): 
        sublists.append(New_RPT[i::8])
    return sublists

def FinalExpansionPermutation(New_RPT, a):
    
    a1, a2,a3,a4,a5,a6,a7,a8 = split_array(RPT, 8) 

    #take the first & last value for each sub-array
    #for 1st array
    r1 = a2[0]
    l1 = a8[3]

 
    #for 2nd array
    r2 = a3[0]
    l2 = a1[3]


    #for 3rd array
    r3 = a4[0]
    l3 = a2[3]


    #for 4th array
    r4 = a5[0]
    l4 = a3[3]


    #for 5th array
    r5 = a6[0]
    l5 = a4[3]


    #for 6th array
    r6 = a7[0]
    l6 = a5[3]

    #for 7th array
    r7 = a8[0]
    l7 = a6[3]


    #for 8th array
    r8 = a1[0]
    l8 = a7[3]

    #update 1st array  
    b1 = np.empty((1, 6))
    b1 = np.append(a1, r1)
    b1 = np.append(l1, b1)



    #update 2nd array
    b2 = np.empty((1, 6))
    b2 = np.append(a2, r2)
    b2 = np.append(l2, b2)



    #update 3rd array
    b3 = np.empty((1, 6))
    b3 = np.append(a3, r3)
    b3 = np.append(l3, b3)



    #update 4th array
    b4 = np.empty((1, 6))
    b4 = np.append(a4, r4)
    b4 = np.append(l4, b4)



    #update 5th array
    b5 = np.empty((1, 6))
    b5 = np.append(a5, r5)
    b5 = np.append(l5, b5)



    #update 6th array
    b6 = np.empty((1, 6))
    b6 = np.append(a6, r6)
    b6 = np.append(l6, b6)


    #update 7th array
    b7 = np.empty((1, 6))
    b7 = np.append(a7, r7)
    b7 = np.append(l7, b7)



    #update 8th array
    b8 = np.empty((1, 6))
    b8 = np.append(a8, r8)
    b8 = np.append(l8, b8)


    #integrate the 8s blocks of Expansion Permutation 
    block=np.append(b1,b2)
    block=np.append(block,b3)
    block=np.append(block,b4)
    block=np.append(block,b5)
    block=np.append(block,b6)
    block=np.append(block,b7)
    block=np.append(block,b8)

    print(block)
    return block
# FinalExpansionPermutation(RPT, 8) 

In [23]:
#XOR between IP's output and Key Trans's output
# PermutedMsg=InitialPermute(IP_Table, BinaryM)
# LPT , RPT =  Permute_Split(PermutedMsg)

# fblock = FinalExpansionPermutation(RPT , 8)
# fkey = FinalKeyTransformation(K)

# XOR_ExP = fblock
# print(XOR_ExP)
# XOR_Key = np.array(list(fkey[0]), dtype=int)


def Xor(XOR_key,XOR_ExP):
  
  for i in range(48):
      
      XOR_ExP[i]=XOR_Key[i]^XOR_ExP[i]
  
  
  return XOR_ExP
# Xor(XOR_Key,XOR_ExP)

**S-Box Substitution**

In [25]:
def FinalS_BoxSubstitution(XOR_Key,XOR_ExP):
  XOR_output = Xor(XOR_Key,XOR_ExP) #48-bit output
  c1,c2,c3,c4,c5,c6,c7,c8= np.array_split(XOR_output, 8) 

  s_box_table_1 = ( [14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
                [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
                [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
                [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13])
  s_box_table_2 = (  [15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
                  [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
                  [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
                  [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9]) 
  s_box_table_3 = (  [10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
                  [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
                  [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
                  [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12])

  s_box_table_4 = (  [7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
                  [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
                  [10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
                  [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14])
    
  s_box_table_5 =(   [2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
                  [14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
                  [4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
                  [11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3])

  s_box_table_6 = (  [12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
                  [10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
                  [9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
                  [4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13])        

  s_box_table_7 =(   [4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
                  [13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
                  [1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
                  [6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12])

  s_box_table_8 = (  [13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
                  [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
                  [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
                  [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11])

  x1=([c1[0],c1[5]])  
  y1=([c1[1],c1[2],c1[3],c1[4]]) 

  strings1_1 = [str(integer) for integer in x1]
  a_string1_1 = "".join(strings1_1)
  strings1_2 = [str(integer) for integer in y1]
  a_string1_2 = "".join(strings1_2)
  row1=int(a_string1_1,2)
  col1=int(a_string1_2,2)

  d1=s_box_table_1[row1][col1]
  d1=bin(d1).replace("0b","")
  x1 = d1[::-1] #this reverses an array
  while len(x1) < 4:
   x1 += '0'
  d1 = x1[::-1]


  x2=([c2[0],c2[5]])  
  y2=([c2[1],c2[2],c2[3],c2[4]]) 

  strings2_1 = [str(integer) for integer in x2]
  a_string2_1 = "".join(strings2_1)
  strings2_2 = [str(integer) for integer in y2]
  a_string2_2 = "".join(strings2_2)
  row2=int(a_string2_1,2)
  col2=int(a_string2_2,2)

  d2=s_box_table_2[row2][col2]

  d2=bin(d2).replace("0b","")
  x2 = d2[::-1] #this reverses an array
  while len(x2) < 4:
   x2 += '0'
  d2 = x2[::-1]

  x3=([c3[0],c3[5]])  
  y3=([c3[1],c3[2],c3[3],c3[4]]) 

  strings3_1 = [str(integer) for integer in x3]
  a_string3_1 = "".join(strings3_1)
  strings3_2 = [str(integer) for integer in y3]
  a_string3_2 = "".join(strings3_2)
  row3=int(a_string3_1,2)
  col3=int(a_string3_2,2)

  d3=s_box_table_3[row3][col3]

  d3=bin(d3).replace("0b","")
  x3 = d3[::-1] #this reverses an array
  while len(x3) < 4:
   x3 += '0'
  d3 = x3[::-1]


  x4=([c4[0],c4[5]])  
  y4=([c4[1],c4[2],c4[3],c4[4]]) 
  strings4_1 = [str(integer) for integer in x4]
  a_string4_1 = "".join(strings4_1)
  strings4_2 = [str(integer) for integer in y4]
  a_string4_2 = "".join(strings4_2)
  row4=int(a_string4_1,2)
  col4=int(a_string4_2,2)
  d4=s_box_table_4[row4][col4]
  d4=bin(d4).replace("0b","")
  x4 = d4[::-1] #this reverses an array
  while len(x4) < 4:
   x4 += '0'
  d4 = x4[::-1]


  x5=([c5[0],c5[5]])  
  y5=([c5[1],c5[2],c5[3],c5[4]]) 

  strings5_1 = [str(integer) for integer in x5]
  a_string5_1 = "".join(strings5_1)
  strings5_2 = [str(integer) for integer in y5]
  a_string5_2 = "".join(strings5_2)
  row5=int(a_string5_1,2)
  col5=int(a_string5_2,2)

  d5=s_box_table_5[row5][col5]

  d5=bin(d5).replace("0b","")
  x5 = d5[::-1] #this reverses an array
  while len(x5) < 4:
   x5 += '0'
  d5 = x5[::-1]

  x6=([c6[0],c6[5]])  
  y6=([c6[1],c6[2],c6[3],c6[4]]) 

  strings6_1 = [str(integer) for integer in x6]
  a_string6_1 = "".join(strings6_1)
  strings6_2 = [str(integer) for integer in y6]
  a_string6_2 = "".join(strings6_2)
  row6=int(a_string6_1,2)
  col6=int(a_string6_2,2)

  d6=s_box_table_6[row6][col6]

  d6=bin(d6).replace("0b","")
  x6 = d6[::-1] #this reverses an array
  while len(x6) < 4:
    x6 += '0'
  d6 = x6[::-1]

  x7=([c7[0],c7[5]])  
  y7=([c7[1],c7[2],c7[3],c7[4]]) 

  strings7_1 = [str(integer) for integer in x7]
  a_string7_1 = "".join(strings7_1)
  strings7_2 = [str(integer) for integer in y7]
  a_string7_2 = "".join(strings7_2)
  row7=int(a_string7_1,2)
  col7=int(a_string7_2,2)

  d7=s_box_table_7[row7][col7]
  d7=bin(d7).replace("0b","")
  x7 = d7[::-1] #this reverses an array7
  while len(x7) < 4:
      x7 += '0'
  d7 = x7[::-1]


  x8=([c8[0],c8[5]])  
  y8=([c8[1],c8[2],c8[3],c8[4]]) 

  strings8_1 = [str(integer) for integer in x8]
  a_string8_1 = "".join(strings8_1)
  strings8_2 = [str(integer) for integer in y8]
  a_string8_2 = "".join(strings8_2)
  row8=int(a_string8_1,2)
  col8=int(a_string8_2,2)

  d8=s_box_table_8[row8][col8]
 
  d8=bin(d8).replace("0b","")
  x8 = d8[::-1] #this reverses an array8
  while len(x8) < 4:
      x8 += '0'
  d8 = x8[::-1]

  output=d1+d2+d3+d4+d5+d6+d7+d8


  S_Box = np.empty(0,dtype=np.int8)



  res = [int(x) for x in output]

  for char in res:
      S_Box = np.append(S_Box,int(char))

  

  out_arr = np.array(S_Box)
  return out_arr
# FinalS_BoxSubstitution(XOR_Key,XOR_ExP)
  

**P-Box Permutation**

In [26]:
# P-Box Permutation Implementation

# This is the given P-Box Table which is used for swapping S-box output 32 bits

P_Box_Table = [ 16,  7, 20, 21,
        29, 12, 28, 17, 
         1, 15, 23, 26, 
         5, 18, 31, 10, 
         2,  8, 24, 14, 
        32, 27,  3,  9, 
        19, 13, 30,  6, 
        22, 11,  4, 25 ]


# This is the method where we rearrange bits of S-Box output (P-Box Permutation)


def FinalP_BoxPermutation(P_Box_Table, S_M):
  New_M = ""  # To add in it the rearranged bits of Input_M
  # add in New_M the value in Input_M[i] (ex: Input_M[58])
  # where the i comes from the P_Box_Table/IP_Table/FP_Table and the - 1 due to starting from 0 index
  for i in P_Box_Table:
		  New_M += str(S_M[int(i)-1])
  


  
  # out_arr = ""  #output from 0 to 31 string
  # for i in S_M:
  #   out_arr += str(i)
  return New_M 






**XOR and Swap**

In [28]:
def update(L,fn,r):

    


    of = FinalP_BoxPermutation(P_Box_Table, fn)
    ol = LPT
    Or = RPT
    # print(of.type)
    # print(L.type)
    # print(r.type)
    # print("reemoooo" ,len(of))
    # print(Or)


    f = [] 
    


    # for i in range(32):
    #   f.append(int(of[i]+""))

   

    f=np.array(list(of), dtype=int)


    r = [] 
    
    for i in Or:
     r.append(i) #from RPT



    l = [] 
    
    for i in ol:
      l.append(i) #LPT



    return l,f,r


def FinalXorandSwap(L,fn,r):

  L,fn,r = update(L,fn,r)

  temp = []

  print("len r", len(r))
  print("len fn", len(fn))
  print("len L", len(L))



  for i in range(32):
      temp.append(r[i])
      r[i]=L[i]^int(fn[i])
       
  L=temp  
  
  
  nl , nr = L , r

  
      
  for i in nr :
      nl.append(i) #RPT

  ff = ""
  
  for i in nl:
    ff += str(i)  

  return ff   #Do i need to output the LPT or not
# print(ff)

  



**16-Round Loop**

In [48]:
FP_Table = ['40 ', '8 ', '48 ', '16 ', '56 ', '24 ', '64 ', '32',
			     '39 ', '7 ', '47 ', '15 ', '55 ', '23 ', '63 ', '31',
			     '38 ', '6 ', '46 ', '14 ',  '54 ', '22 ', '62 ', '30',
			     '37 ', '5 ', '45 ', '13 ', '53 ', '21 ', '61 ', '29',
			     '36 ', '4 ', '44 ', '12 ', '52 ', '20 ', '60 ', '28',
			     '35 ', '3 ', '43 ', '11 ', '51 ', '19 ', '59 ', '27', 
			     '34 ', '2 ', '42 ', '10 ', '50 ', '18 ', '58 ', '26',
			     '33 ', '1 ', '41 ', '9 ', '49 ', '17 ', '57 ', '25']
def FinalPermutation(FP_TABLE, R_M):
    ciphertext = PermutationFunction(FP_TABLE,R_M)
    return ciphertext

# Testing the FinalPermute Method

# RoundedMsg=InitialPermute(IP_Table, BinaryM)
# RoundedMsg
def encrypt(key):
  key="hello"
  key_array=string_to_bit_array(key)
  if len(key_array)<64:
   return "Please enter a longer key "
  else :
        key_array= key_array[:63]
        res_k =  FinalKeyTransformation(key_array)
        for j in range(len(IP_input)):
          plain_text=IP_input[j]
          premuted_msg=InitialPermute(IP_Table,plain_text)
          LPT,RPT = Permute_Split(permuted_msg)
          for i in range(16):
             res_EP = FinalExpansionPermutation(RPT , 8)

             res_S_Box = FinalS_BoxSubstitution(int(res_k[i]),res_EP)

             res_P_Box = FinalP_BoxPermutation(P_Box_Table, res_S_Box)

             res_XOR_and_Swap =  FinalXorandSwap(LPT,res_P_Box,RPT)
        output=FinalPermutation(FP_Table,res_XOR_and_Swap)


**Final Permutation**

In [ ]:
# Final Permutation Implementation

# This is the given FP Table which is used to Inverse Permute the Rounded M bits 



# This is the method where we reverse rearrange bits of Rounded M (FP)




In [ ]:
CBC MODE

In [63]:
FP_Table = ['40 ', '8 ', '48 ', '16 ', '56 ', '24 ', '64 ', '32',
			     '39 ', '7 ', '47 ', '15 ', '55 ', '23 ', '63 ', '31',
			     '38 ', '6 ', '46 ', '14 ',  '54 ', '22 ', '62 ', '30',
			     '37 ', '5 ', '45 ', '13 ', '53 ', '21 ', '61 ', '29',
			     '36 ', '4 ', '44 ', '12 ', '52 ', '20 ', '60 ', '28',
			     '35 ', '3 ', '43 ', '11 ', '51 ', '19 ', '59 ', '27', 
			     '34 ', '2 ', '42 ', '10 ', '50 ', '18 ', '58 ', '26',
			     '33 ', '1 ', '41 ', '9 ', '49 ', '17 ', '57 ', '25']
def FinalPermutation(FP_TABLE, R_M):
    ciphertext = PermutationFunction(FP_TABLE,R_M)
    return ciphertext

# Testing the FinalPermute Method

# RoundedMsg=InitialPermute(IP_Table, BinaryM)
# RoundedMsg
from past.builtins import xrange

def encrypt(key):
  key="hello"
  key_array=string_to_bit_array(key)
  if len(key_array)<64:
   return "Please enter a longer key "
  else :
        key_array= key_array[:63]
        res_k =  FinalKeyTransformation(key_array)
        for j in range(len(IP_input)):
           plain_text=IP_input[j]
           if j==0 :
              for i in range(64):
                CBC_Text[i]=plain_text[i]^key_array[i]
              premuted_msg=InitialPermute(IP_Table,CBC_Text)
              LPT,RPT = Permute_Split(permuted_msg)
           else :
            for i in range(64):
              CBC_Text[i]=output[i]^plain_text[i]
            premuted_msg=InitialPermute(IP_Table,CBC_Text)
            LPT,RPT = Permute_Split(permuted_msg)
            for i in range(16):
             res_EP = FinalExpansionPermutation(RPT , 8)

             res_S_Box = FinalS_BoxSubstitution(int(res_k[i]),res_EP)

             res_P_Box = FinalP_BoxPermutation(P_Box_Table, res_S_Box)

             res_XOR_and_Swap =  FinalXorandSwap(LPT,res_P_Box,RPT)
        output=FinalPermutation(FP_Table,res_XOR_and_Swap)
        bin_chunks = [output[8*i:8*(i+1)] for i in xrange(len(output)//8)]
       
        ints = [int(x, 2) for x in bin_chunks]

        chars = [chr(x) for x in ints]
        outChar = ''.join(chars)

        

# plain_txt=bitarray(string_to_bit_array(text))
# input_2=[]
# pl_1=plain_txt[0:63]
# for i in range(63):
#     # input_2.append(pl_1[i]^int(RoundedMsg[i]))
# print(pl_1)
# print(str(input_2))
#converting binary to ascii



NameError: ignored